In [ ]:
from __future__ import division, print_function
from collections import namedtuple,defaultdict
from bokeh.plotting import figure, output_file, show
from ipywidgets import fixed, interact, interact_manual, interactive
from matplotlib import pyplot as plt
from PIL import Image
from sklearn.manifold import TSNE
from face import face
import face_recognition_models
import pickle
import cv2
import ipywidgets as widgets
import glob
import os
import numpy as np

In [ ]:
def retImg(in_img):
    s = in_img.shape
    x = s[0]
    y = s[1]
    img = np.empty((x, y), dtype=np.uint32)
    view = img.view(dtype=np.uint8).reshape((x, y, 4))
    for i in range(x):
        for j in range(y):
            val = in_img[x-i-1,y-j-1]
            view[i, j, 0] = val[2]
            view[i, j, 1] = val[1]
            view[i, j, 2] = val[0]
            view[i, j, 3] = 255
    return img

In [ ]:
#point this to a directory of directories containing example cropped images of ppl of interest
def make_tasking(lookupdir,outfilename):
    tasking = defaultdict(dict)
    for tdir in glob.glob(lookupdir):
        encodings = []
        pics = []
        print(tdir)
        for qfile in glob.glob(os.path.join(tdir,'*')):
            face_image = cv2.imread(qfile)
            locs = face.face_locations(face_image)
            enc = face.face_encodings(face_image, None)
            if enc and len(enc) >=1:
                print(qfile)
                top, right, bottom, left = locs[0]
                encodings.append(face.face_encodings(face_image, None)[0])
                cv2.rectangle(face_image, (left, top),
                              (right, bottom), (0, 255, 0), 2)
                pics.append(face_image[top:bottom, left:right])
        key = tdir.split('/')[-1]
        tasking[key]['enc']= encodings
        tasking[key]['pic'] = pics
    pickle.dump(tasking,open(outfilename,'wb'))

In [ ]:
#compile the pickles into something to hand to tsne for viz
def make_datastore(people,tasking=None):

    X = list()
    COLOR = list()
    IMG = list()
    dw = list()
    dh = list()

    for p in people:
        dat = people[p]['face_vec']
        COLOR.append('black')
        X.append(dat)
        IMG.append(retImg(people[p]['pic']))
        dw.append(10)
        dh.append(10)
    
    idx=0
    if tasking is not None:
        for p in tasking:
            colors=['red','green','magenta','blue','orange','purple']

            for x in tasking[p]['enc']:
                COLOR.append(colors[idx])
                X.append(x)

            for x in tasking[p]['pic']:
                IMG.append(retImg(x))
                dw.append(10)
                dh.append(10)
            idx +=1
    return(X,COLOR,IMG,dw,dh)

In [ ]:
#plot dots where faces are, black dots for video data, colored dots for people of interest
def tsne_faces(prep,X,COLOR,lr,d):
    model = TSNE(n_components=2, random_state=234)
    model.perplexity = prep
    model.learning_rate = lr
    d['prep']=prep
    d['lr']=lr
    np.set_printoptions(suppress=False)
    out= model.fit_transform(X) 
    # plot the result
    vis_x = out[:, 0]
    vis_y = out[:, 1]

    plt.figure(figsize=(15, 15))
    plt.scatter(vis_x, vis_y, c=COLOR)
    plt.show()
    return (prep,lr)

In [ ]:
#plot pictures blue frame is video data, green frame is where a query is
def tsne_html(datastore,d):
    X,COLOR,IMG,dw,dh = DATASTORE
    model = TSNE(n_components=2, random_state=234)
    model.perplexity=d['prep']
    model.learning_rate = d['lr']
    np.set_printoptions(suppress=False)
    out= model.fit_transform(X) 
    # plot the result
    vis_x = out[:, 0]*50
    vis_y = out[:, 1]*50
    
    p = figure(plot_width=800, plot_height=800, x_range=(-80, 80), y_range=(-80,80))

    p.image_rgba(image=IMG , x=vis_x , y=vis_y , dw=dw , dh=dh )

    show(p)
    #output_file("faces.html")
    

In [ ]:
people = pickle.load(open('persons.pickle','rb'))
tasking = pickle.load(open('find_us.pickle','rb'))

#pretend you have ppl of interest
DATASTORE = make_datastore(people,tasking)

#pretend you just have video processed
#DATASTORE = make_datastore(people)
d = dict()
d['prep']=18
d['lr']=21



In [ ]:
#plot dots
chain = interact(tsne_faces,
         prep=widgets.IntSlider(min=1,max=100,step=1,value=6),
         X=fixed(DATASTORE[0]),
         COLOR=fixed(DATASTORE[1]),
         lr=widgets.IntSlider(min=1,max=1000,step=1,value=70),
         d=fixed(d));

In [ ]:
#plot images
#prep=18, lr=21
tsne_html(DATASTORE,d)